In [ ]:
#build dataframe for data 
DATA = 'prefixed_train.xlsx'
df = pd.read_excel (DATA)
print (df)

In [ ]:
#shuffle data
df = df.sample(frac=1).reset_index(drop=True)
df

In [ ]:
cols=[0,5,6,9]
X = df.drop(df.columns[cols], axis=1)

In [ ]:
X

In [ ]:
cnt_df = pd.read_excel('train_depth.xlsx')

def decide_parents_by_depth(cnt_df, X):
  chemical_to_depth={}
  for idx, row in cnt_df.iterrows():
    if row['Chemical'] not in chemical_to_depth:
      chemical_to_depth[row['Chemical']]=[]
    chemical_to_depth[row['Chemical']] .append(row['Depth'])
    chemical_to_depth[row['Chemical']]=list(set(chemical_to_depth[row['Chemical']]))
  #add depth as a feature
  depth=[]
  for idx,row in X.iterrows(): 
    d = chemical_to_depth[row['Chemical']]
    string_d = [str(elem) for elem in d]
    depth.append(string_d)
  X['Depth']=depth
  depth_X= X[['Depth']].join(X.Depth.str.join('|').str.get_dummies().add_prefix('depth_'))
  X = pd.concat([X,depth_X],axis=1) #merge them together 
  X = X.drop(['Depth'], axis=1)
  X
  return X

def drop_parents_by_depth(cnt_df, not_remove_cols,X):
  #drop the chemicals by the depth
  toRemove=[]
  #not_remove_col=['depth_4','depth_5','depth_6','depth_7','depth_8','depth_9','depth_10']
  not_remove_col = not_remove_cols 
  for idx, row in X.iterrows():
    if row['Appeared']==0 and row['Indexing(T/F)']==0: #added as a parent but is not an indexing chemical
      if row['depth_0']==1 or row['depth_1']==1 or row['depth_2']==1 or row['depth_3']==1:
        remove=True
        for col in not_remove_col: #check if it is not contained to another column
          if row[col]==1:
            remove=False
            break
        if remove is True:
          toRemove.append(idx)

  #drop 
  X= X.drop(X.index[toRemove])
  X = X.loc[:, ~X.columns.str.startswith('depth')]
  return X 

X=decide_parents_by_depth(cnt_df,X)
X
#TO CHANGE:
#not_remove_col=['depth_4','depth_5','depth_6','depth_7','depth_8','depth_9'] NLMCHEM
not_remove_col=['depth_4','depth_5','depth_6','depth_7','depth_8','depth_9','depth_10'] #BC5CDR
X = drop_parents_by_depth(cnt_df,not_remove_col,X )

In [ ]:
y = X['Indexing(T/F)']
X = X.drop(['Chemical','Appeared','Indexing(T/F)'], axis=1) #use the hierarchy prefix instead of the chemical identifier
X

In [ ]:
#Target encoding - after the split 
!pip install --upgrade category_encoders
from category_encoders import TargetEncoder
encoder = TargetEncoder()
#X is training data
X = encoder.fit_transform(X,y)

In [ ]:
#to handle class imbalance
#equally penalize under or over-represented classes in the training set.
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y),y=y)

print("Class Weights:",class_weights)

class_weight = { 0: class_weights[0] , 1: class_weights[1]}

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y,
                                                stratify=y,  #set the class ratio equal as the original training set
                                                test_size=0.20)

In [ ]:
from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf
from keras.models import load_model
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_fscore', patience=5, verbose=1,factor=0.5, min_lr=0.0001)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import SGD,Adam,RMSprop
from keras.regularizers import l2
#creating model
def create_model(optimizer='adam', activation = 'relu',units=1, hidden_layers = 1, regularizer=0.0, learning_rate = 0.1, dropout=False):
  # Initialize the constructor
  model = Sequential()
  # Add an input layer
  model.add(Dense(units,input_dim = X.shape[1], activation=activation,kernel_initializer='he_uniform'))
  # Add one hidden layer
  for i in range(hidden_layers):
    model.add(Dense(units*(i+2),activation=activation,  kernel_regularizer=l2(regularizer)))
    if dropout is True:
      model.add(Dropout(0.2))
  # Add an output layer 
  model.add(Dense(1, activation='sigmoid'))
  #add an optimizer
  if optimizer is 'adam':
    optimizer = Adam(learning_rate=learning_rate)
  elif optimizer is 'sgd':
    optimizer=SGD(learning_rate=lr)
  elif optimizer is 'rmsprop':
    optimizer = RMSprop(learning_rate=learning_rate)
  #compile model
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  return model

In [ ]:
model = create_model(units=20, hidden_layers = 3, dropout=False, learning_rate=0.0003)

In [ ]:
history = model.fit(X_train, y_train, batch_size=64, epochs=15, validation_data=(X_val,y_val), shuffle=True, callbacks=[callback,learning_rate_reduction], class_weight = class_weight)
model.save('nlm+cdr_trained_model.h5') 

In [ ]:
#Evaluation on the dev set 
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

y_pred1 = model.predict(X_test)
y_pred=[]
for elem in y_pred1:
  if elem>0.50:
    y_pred.append([1])
  else:
    y_pred.append([0])


In [ ]:
def get_test_performance(answer,pred):
    #print precision, recall
    print("Precision:",metrics.precision_score(answer, pred))
    print("Recall:",metrics.recall_score(answer, pred))
    print("F1:",metrics.f1_score(answer,pred))
    #printing out confusion matrix 
    print(confusion_matrix(answer,pred))
    print(classification_report(answer,pred))

get_test_performance(y_test,y_pred)